In [2]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf
import os
from tensorflow.keras.callbacks import ModelCheckpoint

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("../Resources/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
run1_df = application_df.drop(columns=['EIN','NAME'])

# Get application_counts
application_counts = run1_df["APPLICATION_TYPE"].value_counts()

# Determine which values to replace if counts are less than 500
replace_application = list(application_counts[application_counts<500].index)

# Replace in dataframe
for app in replace_application:
    run1_df.APPLICATION_TYPE = run1_df.APPLICATION_TYPE.replace(app,"Other")

# Get class_counts
class_counts = run1_df["CLASSIFICATION"].value_counts()

# Determine which values to replace if counts are less than 1800
replace_class = list(class_counts[class_counts<1800].index)

# Replace in dataframe
for cls in replace_class:
    run1_df.CLASSIFICATION = run1_df.CLASSIFICATION.replace(cls,"Other")
run1_df.CLASSIFICATION.value_counts()

# Place to drop variables from the test
run1_df = run1_df.drop(["ASK_AMT","ORGANIZATION","USE_CASE","INCOME_AMT","CLASSIFICATION","STATUS","SPECIAL_CONSIDERATIONS"],1)


In [4]:

# Generate our categorical variable lists
application_cat = run1_df.dtypes[run1_df.dtypes == "object"].index.tolist()

# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(run1_df[application_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(application_cat)

# Merge one-hot encoded features and drop the originals
appli1_df = run1_df.merge(encode_df,left_index=True,right_index=True)
appli1_df = appli1_df.drop(application_cat,1)

# Split our preprocessed data into our features and target arrays
y = appli1_df.IS_SUCCESSFUL.values
X = appli1_df.drop("IS_SUCCESSFUL",1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 100
hidden_nodes_layer2 = 20
#hidden_nodes_layer3 = 35
hidden_nodes_layer4 = 35

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1,input_dim=number_input_features, activation="tanh")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="tanh"))

# Third hidden layer
#nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Fourth hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer4, activation="softplus"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 100)               1600      
_________________________________________________________________
dense_1 (Dense)              (None, 20)                2020      
_________________________________________________________________
dense_2 (Dense)              (None, 35)                735       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 36        
Total params: 4,391
Trainable params: 4,391
Non-trainable params: 0
_________________________________________________________________


In [5]:

# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/run1_wts.{epoch:02d}.hdf5"

# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Create a callback that saves the model's weights every epoch
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq='epoch',
    period = 5)

In [6]:
# Train the model
fit_model = nn.fit(X_train,y_train,epochs=100,callbacks=[cp_callback])

# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 1/100
804/804 [==============================] - 1s 565us/step - loss: 0.5947 - accuracy: 0.7123
Epoch 2/100
804/804 [==============================] - 0s 559us/step - loss: 0.5858 - accuracy: 0.7206
Epoch 3/100
804/804 [==============================] - 0s 557us/step - loss: 0.5850 - accuracy: 0.7201
Epoch 4/100
804/804 [==============================] - 0s 542us/step - loss: 0.5845 - accuracy: 0.7205
Epoch 5/100
804/804 [==============================] - 0s 550us/step - loss: 0.5842 - accuracy: 0.7203

Epoch 00005: saving model to checkpoints\run1_wts.05.hdf5
Epoch 6/100
804/804 [==============================] - 0s 546us/step - loss: 0.5844 - accuracy: 0.7205
Epoch 7/100
804/804 [==============================] - 0s 552us/step - loss: 0.5838 - accuracy: 0.7198
Epoch 8/100
804/804 [==============================] - 0s 595us/step - loss: 0.5841 - accuracy: 0.7203
Epoch 9/100
804/804 [==============================] - 0s 558us/step - loss: 0.5837 - accuracy: 0.7200
Epoch 10/100
8

804/804 [==============================] - 0s 559us/step - loss: 0.5824 - accuracy: 0.7204
Epoch 72/100
804/804 [==============================] - 0s 558us/step - loss: 0.5824 - accuracy: 0.7207
Epoch 73/100
804/804 [==============================] - 0s 545us/step - loss: 0.5822 - accuracy: 0.7207
Epoch 74/100
804/804 [==============================] - 0s 556us/step - loss: 0.5826 - accuracy: 0.7203
Epoch 75/100
804/804 [==============================] - 0s 550us/step - loss: 0.5823 - accuracy: 0.7200

Epoch 00075: saving model to checkpoints\run1_wts.75.hdf5
Epoch 76/100
804/804 [==============================] - 0s 554us/step - loss: 0.5825 - accuracy: 0.7206
Epoch 77/100
804/804 [==============================] - 0s 554us/step - loss: 0.5825 - accuracy: 0.7210
Epoch 78/100
804/804 [==============================] - 0s 583us/step - loss: 0.5825 - accuracy: 0.7206
Epoch 79/100
804/804 [==============================] - 0s 551us/step - loss: 0.5826 - accuracy: 0.7204
Epoch 80/100
804/8

In [387]:
# Export our model to HDF5 file
nn.save("AlphabetSoupCharity_Optimization.h5")